In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from zytholic_project.reviews_cleaning import *
from zytholic_project.base_model import BaseModel

In [ ]:
!pip install nltk

In [ ]:
#reviews_sorted()

from IPython.display import Audio
sound_file = '/Users/alix/Music/iTunes/iTunes Media/Music/Powerglove/Total Pwnage/01 Tetris.mp3' #un son dans votre ordi ou une url
Audio(sound_file, autoplay=True)

In [ ]:
text_df = pd.read_csv('/Users/alix/code/ltexp1998/zytholic_project/raw_data/reviews_top_beer_concatenated.csv')

In [ ]:
#text_df['text'] = text_df['text'].apply(preprocessing)

text_df.to_csv('/Users/alix/code/ltexp1998/zytholic_project/raw_data/reviews_top_beer_preprocessed.csv', index=False)




In [21]:
text_df = pd.read_csv(
    '/Users/alix/code/ltexp1998/zytholic_project/raw_data/reviews_top_beer_preprocessed.csv', 
    index_col = 'Unnamed: 0'
)
text_df = text_df.dropna()

encoded = reviews_featuring(text_df, 20)

encoded.drop('text', axis=1, inplace=True)

new = pd.merge(a.working_df, encoded, on='beer_id', how='left')

new.shape

In [33]:
new.columns

Index([          'name', 'original_style',        'brewery',            'abv',
           'ave rating',        'min ibu',        'max ibu',    'astringency',
                 'body',        'alcohol',         'bitter',          'sweet',
                 'sour',          'salty',         'fruits',          'hoppy',
               'spices',          'malty',        'beer_id',          'state',
              'country',        'retired',          'style',           'milk',
                  'old',           'dark',           'wild',           'pale',
                  'red',       'imperial',                0,                1,
                      2,                3,                4,                5,
                      6,                7,                8,                9,
                     10,               11,               12,               13,
                     14,               15,               16,               17,
                     18,               19],
      dt

# Comparing results with tfidif

In [ ]:
import numpy as np
import pandas as pd

from sklearn import set_config; set_config(display='diagram')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.compose import make_column_transformer

class TextFeaturesPrediction():
    def __init__(self, working_df):
        self.working_df = self.reformat_styles(working_df)
        
        
    def reformat_styles(self, working_df, ohe=True):
        """
        Simplify the columns 'style' of an input DF
        Converts various features insides style name to OHE features
        """

        # Get matching table for styles names and format it
        style_csv = pd.read_csv('../assets/style_convert.csv')
        style_csv = style_csv[['Converted', 'Simplified']]

        # creation of a dictionary to replace automatically
        style_dict = style_csv.set_index('Converted').to_dict()
        style_dict = style_dict['Simplified']
        style_dict

        #styles_test = working_df[['style']].drop_duplicates()
        working_df['simple_style'] = working_df['style'].replace(style_dict)

        # One-Hot-Encoding of featrues_to_implement
        features_to_implement = ['milk', 'old', 'dark', 'wild', 'pale', 'red', 'imperial']
        if ohe:
            for feat in features_to_implement:
                working_df[feat] = [1 if feat in elm.lower() else 0 for elm in working_df['style']]

        working_df.rename(columns={'style':'original_style', 'simple_style':'style'}, inplace=True)
        return working_df


    # ----------------- PIPELINES ------------------------------
    def set_preprocess_pipeline(self):
        """Create preprocessing pipeline for the data frame"""
        pipe_style_country = make_pipeline(OneHotEncoder(sparse=False,
                                                        handle_unknown='ignore'))
        num_features_top = make_pipeline(MinMaxScaler())
        # pipe_state = make_pipeline(
        #     SimpleImputer(strategy='constant', fill_value=''),
        #     OneHotEncoder(sparse=False, handle_unknown='ignore')
        # )

        # list of numeric columns for min-max scaler
        tastes_features = self.working_df.select_dtypes(np.number).columns

        # Combine the preprocessing pipelines
        preprocess = make_column_transformer(
            ## Uncomment pipe_state and activate
            #(pipe_state, ['state']),
            (pipe_style_country, ['style',
                                  ##'country' # Deactivated beacause decreases recommendation
                                  ## accuracy, as to many OHE variables relate to geography
                                ]),
            (num_features_top, tastes_features)
        )
        self.preprocess = preprocess
        return self

    def process_data(self):
        """Split dataset before fitting the model"""
        X_train, X_test = train_test_split(self.working_df,
                                           test_size=0.25,
                                           stratify=self.working_df['style'])
        self.X_train = X_train
        self.X_test = X_test
        self.preprocess.fit(X_train)
        self.X_train_proc = self.preprocess.transform(X_train)
        self.X_test_proc = self.preprocess.transform(X_test)
        return self

In [ ]:
temp = wording_df.fillna(value=0).drop(['key', 'beer_id', 'abv', 'min ibu', 'max ibu'], axis=1).reset_index(drop=True)
model_features = TextFeaturesPrediction(temp)
model_features.set_preprocess_pipeline()
model_features.process_data()

In [ ]:
results.groupby('style')[['matching_percent', 'matching_substyle']].agg('describe')

In [ ]:
results

In [63]:
a = BaseModel().get_data()
#b = BaseModel().get_data()
a.working_df.shape, b.working_df.shape

((4341, 30), (4341, 30))

In [64]:
a.add_text_features()

In [65]:
a.set_preprocess_pipeline()
a.process_data()

In [66]:
a.X_test_proc.shape

(1086, 82)

In [67]:
from zytholic_project.evaluate import test_prediction
results = test_prediction(a, similarity='cosine')

In [68]:
results.groupby('style')[['matching_percent','matching_substyle']].agg('describe')

matching_percent                            \
                                           count   mean  std    min    25%   
style                                                                        
Ale                                         73.0  100.0  0.0  100.0  100.0   
Ale Dark                                    50.0  100.0  0.0  100.0  100.0   
Ale Old                                     11.0  100.0  0.0  100.0  100.0   
Ale Pale                                    41.0  100.0  0.0  100.0  100.0   
Ale Red                                     32.0  100.0  0.0  100.0  100.0   
Ale Strong                                  41.0  100.0  0.0  100.0  100.0   
Altbier                                     11.0  100.0  0.0  100.0  100.0   
Barleywine                                  20.0  100.0  0.0  100.0  100.0   
Bitter                                      17.0  100.0  0.0  100.0  100.0   
Bière de Champagne / Bière Brut              9.0  100.0  0.0  100.0  100.0   
Bière de Garde                              11.0  100.0  0.0  100.0  100.0   
Bock                                        47.0  100.0  0.0  100.0  100.0   
Braggot                                     10.0  100.0  0.0  100.0  100.0   
Brett                                       11.0  100.0  0.0  100.0  100.0   
Dubbel                                      11.0  100.0  0.0  100.0  100.0   
Fruit and Field Beer                         9.0  100.0  0.0  100.0  100.0   
Happoshu                                    10.0  100.0  0.0  100.0  100.0   
Herb and Spice Beer                         10.0  100.0  0.0  100.0  100.0   
IPA                                         54.0  100.0  0.0  100.0  100.0   
Kvass                                        9.0  100.0  0.0  100.0  100.0   
Kölsch                                       9.0  100.0  0.0  100.0  100.0   
Lager                                      169.0  100.0  0.0  100.0  100.0   
Lambic                                      24.0  100.0  0.0  100.0  100.0   
Low Alcohol Beer                             8.0  100.0  0.0  100.0  100.0   
NEIPA                                       11.0  100.0  0.0  100.0  100.0   
Pilsner                                     31.0  100.0  0.0  100.0  100.0   
Porter                                      45.0  100.0  0.0  100.0  100.0   
Pumpkin Beer                                 9.0  100.0  0.0  100.0  100.0   
Quadrupel                                   11.0  100.0  0.0  100.0  100.0   
Rice                                         9.0  100.0  0.0  100.0  100.0   
Rye                                         22.0  100.0  0.0  100.0  100.0   
Sahti                                        9.0  100.0  0.0  100.0  100.0   
Saison                                      10.0  100.0  0.0  100.0  100.0   
Smoked                                      22.0  100.0  0.0  100.0  100.0   
Sour                                        41.0  100.0  0.0  100.0  100.0   
Steam Beer                                  11.0  100.0  0.0  100.0  100.0   
Stout                                       81.0  100.0  0.0  100.0  100.0   
Tripel                                      10.0  100.0  0.0  100.0  100.0   
Wheat                                       58.0  100.0  0.0  100.0  100.0   
Winter Warmer                                9.0  100.0  0.0  100.0  100.0   

                                                     matching_substyle  \
                                   50%    75%    max             count   
style                                                                    
Ale                              100.0  100.0  100.0              73.0   
Ale Dark                         100.0  100.0  100.0              50.0   
Ale Old                          100.0  100.0  100.0              11.0   
Ale Pale                         100.0  100.0  100.0              41.0   
Ale Red                          100.0  100.0  100.0              32.0   
Ale Strong                       100.0  100.0  100.0              41.0   
Altbier    